# Simulations with reinforcement learning

In this notebook you can run simulations with 5 different reinforcement learning algorithms. There are different example buildings that can be simulated.

<br>


In [1]:
import sinergym
from sinergym.utils.callbacks import LoggerEvalCallback, LoggerCallback
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import LoggerWrapper, NormalizeObservation
from sinergym.utils.constants import RANGES_5ZONE
from datetime import datetime
import gym
from stable_baselines3 import DQN, DDPG, PPO, A2C, SAC, TD3


from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
from stable_baselines3.common.logger import configure

from math import exp
from typing import Any, Dict, List, Tuple, Union
from gym import Env

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


# Reward functions

In [2]:
"""Implementation of reward functions."""


class BaseReward(object):

    def __init__(self, env):
        """
        Base reward class.

        All reward functions should inherit from this class.

        Args:
            env (Env): Gym environment.
        """
        self.env = env

    def __call__(self):
        """Method for calculating the reward function."""
        raise NotImplementedError(
            "Reward class must have a `__call__` method.")


class MyLinearReward(BaseReward):

    def __init__(
        self,
        env: Env,
        temperature_variable: Union[str, list],
        energy_variable: str,
        range_comfort_winter: Tuple[int, int],
        range_comfort_summer: Tuple[int, int],
        summer_start: Tuple[int, int] = (6, 1),
        summer_final: Tuple[int, int] = (9, 30),
        energy_weight: float = 0.6,
        #energy_weight: float = 1,
        lambda_energy: float = 0.003,
        #lambda_energy: float = 1,
        lambda_temperature: float = 50
    ):
        """
        Linear reward function.

        It considers the energy consumption and the absolute difference to temperature comfort.

        .. math::
            R = - W * lambda_E * power - (1 - W) * lambda_T * (max(T - T_{low}, 0) + max(T_{up} - T, 0))

        Args:
            env (Env): Gym environment.
            temperature_variable (Union[str, list]): Name(s) of the temperature variable(s).
            energy_variable (str): Name of the energy/power variable.
            range_comfort_winter (Tuple[int,int]): Temperature comfort range for cold season. Depends on environment you are using.
            range_comfort_summer (Tuple[int,int]): Temperature comfort range for hot season. Depends on environment you are using.
            summer_start (Tuple[int,int]): Summer session tuple with month and day start. Defaults to (6,1).
            summer_final (Tuple[int,int]): Summer session tuple with month and day end. defaults to (9,30).
            energy_weight (float, optional): Weight given to the energy term. Defaults to 0.5.
            lambda_energy (float, optional): Constant for removing dimensions from power(1/W). Defaults to 1e-4.
            lambda_temperature (float, optional): Constant for removing dimensions from temperature(1/C). Defaults to 1.0.
        """

        super(MyLinearReward, self).__init__(env)

        # Name of the variables
        self.temp_name = temperature_variable
        self.energy_name = energy_variable

        # Reward parameters
        self.range_comfort_winter = range_comfort_winter
        self.range_comfort_summer = range_comfort_summer
        self.W_energy = energy_weight
        self.lambda_energy = lambda_energy
        self.lambda_temp = lambda_temperature

        # Summer period
        self.summer_start = summer_start  # (month,day)
        self.summer_final = summer_final  # (month,day)

    def __call__(self) -> Tuple[float, Dict[str, Any]]:
        """
        Calculate the reward function.

        Returns:
            Tuple[float, Dict[str, Any]]: Reward value and dictionary with their individual components.
        """
        # Current observation
        obs_dict = self.env.obs_dict.copy()

        # Energy term
        #reward_energy = - self.lambda_energy * obs_dict[self.energy_name]
        reward_energy = - (self.lambda_energy * obs_dict[self.energy_name])

        # Comfort
        comfort, temps = self._get_comfort(obs_dict)

        if comfort == 0:
            reward_comfort = 5
        else:
            reward_comfort = - self.lambda_temp * comfort
        # Weighted sum of both terms
        reward = self.W_energy * reward_energy + \
            (1.0 - self.W_energy) * reward_comfort

        reward_terms = {
            'reward_energy': reward_energy,
            'total_energy': obs_dict[self.energy_name],
            'reward_comfort': reward_comfort,
            'abs_comfort': comfort,
            'temperatures': temps
        }

        return reward, reward_terms

    def _get_comfort(self,
                     obs_dict: Dict[str,
                                    Any]) -> Tuple[float,
                                                   List[float]]:
        """Calculate the comfort term of the reward.

        Returns:
            Tuple[float, List[float]]: comfort penalty and List with temperatures used.
        """

        hour = obs_dict["hour"]
        month = obs_dict['month']
        day = obs_dict['day']
        year = obs_dict['year']
        current_dt = datetime(year, month, day)

        # Periods
        summer_start_date = datetime(
            year,
            self.summer_start[0],
            self.summer_start[1])
        summer_final_date = datetime(
            year,
            self.summer_final[0],
            self.summer_final[1])



        if current_dt >= summer_start_date and current_dt <= summer_final_date:
            if current_dt.weekday() >= 5 or hour not in range(8,19):
                temp_range = (15,30)
            else:
                temp_range = self.range_comfort_summer 
        else:
            if current_dt.weekday() >= 5 or hour not in range(8,19):
                temp_range = (15,30)
            else:
                temp_range = self.range_comfort_winter


        temps = [v for k, v in obs_dict.items() if k in self.temp_name]
        comfort = 0.0
        for T in temps:
            if T < temp_range[0] or T > temp_range[1]:
                comfort += min(abs(temp_range[0] - T), abs(T - temp_range[1]))
  
 

        return comfort, temps


class MyExpReward(MyLinearReward):

    def __init__(
        self,
        env: Env,
        temperature_variable: Union[str, list],
        energy_variable: str,
        range_comfort_winter: Tuple[int, int],
        range_comfort_summer: Tuple[int, int],
        summer_start: Tuple[int, int] = (6, 1),
        summer_final: Tuple[int, int] = (9, 30),
        #changes from 0.5 to 0.7
        energy_weight: float = 1,
        lambda_energy: float = 1e-4,
        #lambda_energy: float = 1,
        lambda_temperature: float = 1
    ):
        """
        Reward considering exponential absolute difference to temperature comfort.

        .. math::
            R = - W * lambda_E * power - (1 - W) * lambda_T * exp( (max(T - T_{low}, 0) + max(T_{up} - T, 0)) )

        Args:
            env (Env): Gym environment.
            temperature_variable (Union[str, list]): Name(s) of the temperature variable(s).
            energy_variable (str): Name of the energy/power variable.
            range_comfort_winter (Tuple[int,int]): Temperature comfort range for cold season. Depends on environment you are using.
            range_comfort_summer (Tuple[int,int]): Temperature comfort range for hot season. Depends on environment you are using.
            summer_start (Tuple[int,int]): Summer session tuple with month and day start. Defaults to (6,1).
            summer_final (Tuple[int,int]): Summer session tuple with month and day end. defaults to (9,30).
            energy_weight (float, optional): Weight given to the energy term. Defaults to 0.5.
            lambda_energy (float, optional): Constant for removing dimensions from power(1/W). Defaults to 1e-4.
            lambda_temperature (float, optional): Constant for removing dimensions from temperature(1/C). Defaults to 1.0.
        """

        super(MyExpReward, self).__init__(
            env,
            temperature_variable,
            energy_variable,
            range_comfort_winter,
            range_comfort_summer,
            summer_start,
            summer_final,
            energy_weight,
            lambda_energy,
            lambda_temperature
        )

    def _get_comfort(self,
                     obs_dict: Dict[str,
                                    Any]) -> Tuple[float,
                                                   List[float]]:
        """Calculate the comfort term of the reward.

        Returns:
            Tuple[float, List[float]]: comfort penalty and List with temperatures used.
        """

        hour = obs_dict["hour"]
        month = obs_dict['month']
        day = obs_dict['day']
        year = obs_dict['year']
        current_dt = datetime(year, month, day)

        # Periods
        summer_start_date = datetime(
            year,
            self.summer_start[0],
            self.summer_start[1])
        summer_final_date = datetime(
            year,
            self.summer_final[0],
            self.summer_final[1])

        # if current_dt >= summer_start_date and current_dt <= summer_final_date:
        #     temp_range = self.range_comfort_summer 
        # else:
        #     temp_range = self.range_comfort_winter
        if current_dt >= summer_start_date and current_dt <= summer_final_date:
            if current_dt.weekday() >= 5 or hour not in range(8,19):
                temp_range = (15,30)
            else:
                temp_range = self.range_comfort_summer 
        else:
            if current_dt.weekday() >= 5 or hour not in range(8,19):
                temp_range = (15,30)
            else:
                temp_range = self.range_comfort_winter


        temps = [v for k, v in obs_dict.items() if k in self.temp_name]
        comfort = 0.0
        for T in temps:
            if T < temp_range[0] or T > temp_range[1]:
                comfort += exp(min(abs(temp_range[0] - T),
                                   abs(T - temp_range[1])))

            # else:
            #     comfort -= 5

        return comfort, temps


class MyHourlyExpReward(MyExpReward):

    def __init__(
        self,
        env: Env,
        temperature_variable: Union[str, list],
        energy_variable: str,
        # range_comfort_winter: Tuple[int, int],
        # range_comfort_summer: Tuple[int, int],
        range_comfort_winter = (20,23),
        range_comfort_summer = (23,26),
        summer_start: Tuple[int, int] = (6, 1),
        summer_final: Tuple[int, int] = (9, 30),
        min_energy_weight: float = 0.3,
        #default energy lambda = 1
        lambda_energy: float = 2e-4,
        lambda_temperature: float = 2,
        range_comfort_hours: tuple = (8, 19)
    ):
        """
        Linear reward function with a time-dependent weight for consumption and energy terms.

        Args:
            env (Env): Gym environment.
            temperature_variable (Union[str, list]): Name(s) of the temperature variable(s).
            energy_variable (str): Name of the energy/power variable.
            range_comfort_winter (Tuple[int,int]): Temperature comfort range for cold season. Depends on environment you are using.
            range_comfort_summer (Tuple[int,int]): Temperature comfort range for hot season. Depends on environment you are using.
            summer_start (Tuple[int,int]): Summer session tuple with month and day start. Defaults to (6,1).
            summer_final (Tuple[int,int]): Summer session tuple with month and day end. defaults to (9,30).
            min_energy_weight (float, optional): Minimum weight given to the energy term. Defaults to 0.5.
            lambda_energy (float, optional): Constant for removing dimensions from power(1/W). Defaults to 1e-4.
            lambda_temperature (float, optional): Constant for removing dimensions from temperature(1/C). Defaults to 1.0.
            range_comfort_hours (tuple, optional): Hours where thermal comfort is considered. Defaults to (9, 19).
        """

        super(MyHourlyExpReward, self).__init__(
            env,
            temperature_variable,
            energy_variable,
            range_comfort_winter,
            range_comfort_summer,
            summer_start,
            summer_final,
            min_energy_weight,
            lambda_energy,
            lambda_temperature
        )



        # Reward parameters
        self.range_comfort_hours = range_comfort_hours

    def __call__(self) -> Tuple[float, Dict[str, Any]]:
        """Calculate the reward function.

        Returns:
            Tuple[float, Dict[str, Any]]: Reward and dict with reward terms.
            """
        # Current observation
        obs_dict = self.env.obs_dict.copy()

        # Energy term
        #reward_energy = - self.lambda_energy * obs_dict[self.energy_name]
        reward_energy = - self.lambda_energy * obs_dict['Facility Total HVAC Electricity Demand Rate(Whole Building)']
        # Comfort
        comfort, temps = self._get_comfort(obs_dict)

        if comfort == 0:
            reward_comfort = 0
        else:   
            reward_comfort = - self.lambda_temp * comfort

        # Determine energy weight depending on the hour
        hour = obs_dict['hour']
        if hour >= self.range_comfort_hours[0] and hour <= self.range_comfort_hours[1]:
            weight = self.W_energy
        else:
            weight = 1


        # Weighted sum of both terms
        reward = weight * reward_energy + (1.0 - weight) * reward_comfort

        reward_terms = {
            'reward_energy': reward_energy,
            'total_energy': obs_dict[self.energy_name],
            'reward_comfort': reward_comfort,
            'temperatures': temps
        }

        return reward, reward_terms

class MyHourlyLinearReward(MyLinearReward):

    def __init__(
        self,
        env: Env,
        temperature_variable: Union[str, list],
        energy_variable: str,
        range_comfort_winter: Tuple[int, int],
        range_comfort_summer: Tuple[int, int],
        summer_start: Tuple[int, int] = (6, 1),
        summer_final: Tuple[int, int] = (9, 30),
        min_energy_weight: float = 0.3,
        lambda_energy: float = 2e-4,
        lambda_temperature: float = 2,
        range_comfort_hours: tuple = (8, 19),
    ):
        """
        Linear reward function with a time-dependent weight for consumption and energy terms.

        Args:
            env (Env): Gym environment.
            temperature_variable (Union[str, list]): Name(s) of the temperature variable(s).
            energy_variable (str): Name of the energy/power variable.
            range_comfort_winter (Tuple[int,int]): Temperature comfort range for cold season. Depends on environment you are using.
            range_comfort_summer (Tuple[int,int]): Temperature comfort range for hot season. Depends on environment you are using.
            summer_start (Tuple[int,int]): Summer session tuple with month and day start. Defaults to (6,1).
            summer_final (Tuple[int,int]): Summer session tuple with month and day end. defaults to (9,30).
            min_energy_weight (float, optional): Minimum weight given to the energy term. Defaults to 0.5.
            lambda_energy (float, optional): Constant for removing dimensions from power(1/W). Defaults to 1e-4.
            lambda_temperature (float, optional): Constant for removing dimensions from temperature(1/C). Defaults to 1.0.
            range_comfort_hours (tuple, optional): Hours where thermal comfort is considered. Defaults to (9, 19).
        """

        super(MyHourlyLinearReward, self).__init__(
            env,
            temperature_variable,
            energy_variable,
            range_comfort_winter,
            range_comfort_summer,
            summer_start,
            summer_final,
            min_energy_weight,
            lambda_energy,
            lambda_temperature
        )

        # Reward parameters
        self.range_comfort_hours = range_comfort_hours

    def __call__(self) -> Tuple[float, Dict[str, Any]]:
        """Calculate the reward function.

        Returns:
            Tuple[float, Dict[str, Any]]: Reward and dict with reward terms.
            """
        # Current observation
        obs_dict = self.env.obs_dict.copy()

        # Energy term
        reward_energy = - self.lambda_energy * obs_dict[self.energy_name]

        # Comfort
        comfort, temps = self._get_comfort(obs_dict)

        if comfort == 0:
            reward_comfort = 0
        else:
            reward_comfort = - self.lambda_temp * (comfort)**2

        # Determine energy weight depending on the hour
        hour = obs_dict['hour']
        if hour >= self.range_comfort_hours[0] and hour <= self.range_comfort_hours[1]:
            weight = self.W_energy
        else:
            weight = 1


        # Weighted sum of both terms
        reward = weight * reward_energy + (1.0 - weight) * reward_comfort

        reward_terms = {
            'reward_energy': reward_energy,
            'total_energy': obs_dict[self.energy_name],
            'reward_comfort': reward_comfort,
            'temperatures': temps
        }

        return reward, reward_terms



# create the environment and perform RL



In [3]:
#environment = "Eplus-demo-v1"
environment  = "Eplus-5Zone-hot-continuous-v1"
#environment  = "Eplus-office-hot-continuous-v1"
#environment = "Eplus-5Zone-mixed- continuous-v1"
#weather = "USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw"
#weather = "NLD_Amsterdam.epw"
weather = "ESP_Granada.084190_SWEC.epw"


episodes = 20
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')

#choose the simulation period
begin_day = 1
begin_month = 1
begin_year = 2021
end_day = 1
end_month = 1
end_year = 2022

# register run name
name = F"{environment}-episodes_{episodes}({experiment_date})"


# Set to one month only to reduce running time
extra_params={'timesteps_per_hour' : 4,
              'runperiod' : (begin_day,begin_month,begin_year,end_day,end_month,end_year)}

new_observation_variables=[
    'Site Outdoor Air Drybulb Temperature(Environment)',
    'Site Diffuse Solar Radiation Rate per Area(Environment)',
     'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
     'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
    # 'Zone Thermostat Heating Setpoint Temperature(SPACE2-1)',
    # 'Zone Thermostat Cooling Setpoint Temperature(SPACE2-1)',
    #"Zone Mean Radiant Temperature(SPACE1-1)"
    'Zone Air Temperature(SPACE1-1)',
    #'Zone People Occupant Count(SPACE1-1)',
    'Facility Total HVAC Electricity Demand Rate(Whole Building)']

new_observation_space = gym.spaces.Box(
    low=-5e6,
    high=5e6,
    shape=(len(new_observation_variables) + 4,),
    dtype=np.float32)

new_action_variables = [
    'Heating_Setpoint_RL',
    'Cooling_Setpoint_RL'
]


new_action_space=gym.spaces.Box(
    low=np.array([15.0, 22.5]),
    high=np.array([22.5, 30.0]),
    shape=(2,),
    dtype=np.float32
)
new_action_definition={
    'Htg-SetP-Sch': {'name': 'Heating_Setpoint_RL', 'initial_value': 15},
    'Clg-SetP-Sch': {'name': 'Cooling_Setpoint_RL', 'initial_value': 30}}

env = gym.make(environment, 
                weather_file = weather,
                reward = MyHourlyExpReward, 
                config_params = extra_params,
                observation_variables = new_observation_variables,
                observation_space = new_observation_space,
                action_variables=new_action_variables,
                action_space=new_action_space,
                action_definition=new_action_definition)



#env = LoggerWrapper(NormalizeObservation(env, ranges = RANGES_5ZONE))
env = LoggerWrapper(env)

tensorboard_path='./tensorboard_log'

#choose the algorithm and parameters you want to use
model = DDPG('MlpPolicy', env, verbose=1, tensorboard_log=tensorboard_path,
    batch_size=5000,
    learning_rate=0.001)


n_timesteps_episode = env.simulator._eplus_one_epi_len / env.simulator._eplus_run_stepsize

env_vec = DummyVecEnv([lambda: env])

callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    env_vec,
    best_model_save_path='best_model/' + name + '/',
    log_path='best_model/' + name + '/',
    eval_freq=n_timesteps_episode * 2,
    deterministic=True,
    render=False,
    n_eval_episodes=2)
callbacks.append(eval_callback)

# Set up Evaluation and saving best model
log_callback = LoggerCallback(True)
callbacks.append(log_callback)
# lets change default dir for TensorboardFormatLogger only
tb_path = tensorboard_path + '/' + name
new_logger = configure(tb_path, ["tensorboard"])
model.set_logger(new_logger)

callback = CallbackList(callbacks)

timesteps = episodes * n_timesteps_episode

model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)

model.save(env.simulator._env_working_dir_parent + '/' + name)

env.reset()

env.close()

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


[2023-02-09 22:55:49,635] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-02-09 22:55:49,643] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-02-09 22:55:49,675] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-02-09 22:55:49,703] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-02-09 22:55:49,715] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2023-02-09 22:55:50.060004: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 22:55:52.056445: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-09 22:55:59.050297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-09 22:55:59.050547: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

[2023-02-09 22:56:03,681] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-02-09 22:56:03,942] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res2/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[2023-02-09 23:02:55,409] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-02-09 23:02:55,417] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-02-09 23:02:55,665] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res2/Eplus-env-sub_run2
[2023-02-10 02:26:18,476] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-02-10 02:26:18,491] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-02-10 02:26:18,787] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res2/Eplus-env-sub_run3
[2023-02-10 02:26:53,672] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyP

KeyboardInterrupt: 

In [4]:
from pprint import pprint

env = gym.make("Eplus-5Zone-hot-continuous-v1")
schedulers=env.get_schedulers()
pprint(schedulers)
actuators=env.get_schedulers(path='./testcheck.xlsx')

[2023-02-07 09:18:38,709] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-02-07 09:18:38,709] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-02-07 09:18:38,815] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-02-07 09:18:38,815] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-02-07 09:18:38,867] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-02-07 09:18:38,867] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-02-07 09:18:38,975] EPLUS_ENV_5Zone-hot-continuous-v1_Ma

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [12]:
env = gym.make("hoi")
print(env.get_zones())
env.close()

Error: Attempted to look up malformed environment ID: b'hoi'. (Currently all IDs must be of the form ^(?:[\w:-]+\/)?([\w:.-]+)-v(\d+)$.)